In [ ]:
#imports
import csv
import numpy as np
import string
import pandas as pd
import re
from string import digits

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import spacy
en_core = spacy.load('en_core_web_sm')
nlp_lg = spacy.load('en_core_web_lg')
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier


In [ ]:
#binary
data = pd.read_csv("tweet_emotions.csv")
count = 0
for index, sent in enumerate(data.sentiment):
    if sent == "fun" or sent == "enthusiasm" or sent == "love" or sent == "happiness" or sent =="relief":
        data.loc[index,['sentiment']] = "positive"
    elif sent == "hate" or sent == "anger" or sent == "empty" or sent == "worry" or sent == "sadness" or sent == "boredom":
        data.loc[index,['sentiment']] = "negative"
    else:
        #if count % 6 != 0:
        data.drop([index], axis=0, inplace=True)

data.to_csv('tweet_emotions_binary.csv')

In [ ]:
data
data['sentiment'].value_counts()

In [ ]:
#three classes
data = pd.read_csv("tweet_emotions.csv")
count = 0
for index, sent in enumerate(data.sentiment):
    if sent == "fun" or sent == "enthusiasm" or sent == "love" or sent == "happiness" or sent =="relief":
        data.loc[index,['sentiment']] = "positive"
    elif sent == "hate" or sent == "anger" or sent == "empty" or sent == "worry" or sent == "sadness" or sent == "boredom":
        data.loc[index,['sentiment']] = "negative"
    elif sent == "surprise":
        data.drop([index], axis=0, inplace=True)
data.to_csv('tweet_emotions_three_classes.csv')

In [ ]:
data
data['sentiment'].value_counts()





Data cleaning functions

In [ ]:

def remove_punctuation(line_text):
    line_text_without_punct = re.sub(r'[^\w\s]', '', line_text)
    return line_text_without_punct


def remove_numbers(line_text):
    table = str.maketrans('', '', digits)
    line_text_without_nums = line_text.translate(table)
    return line_text_without_nums

def remove_url(line_text):
    #(https?:\/\/) matches http:// or https://
    #(\s)* optional whitespaces
    #(www\.)? optionally matches www.
    # (\s)* optionally matches whitespaces
    #'((\w|\s)+\.)* matches 0 or more of one or more word characters followed by a period
    #([\w\-\s]+\/)* matches 0 or more of one or more words(or a dash or a space) followed by '\'
    #([\w\-]+) any remaining path at the end of the url followed by an optional ending
    #((\?)?[\w\s]*=\s*[\w\%&]*)* matches ending query params (even with white spaces,etc)

    line_text_without_url = re.sub(r'(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*', '', line_text)
    return line_text_without_url

def remove_username(line_text):
    line_text_without_username = re.sub("([@#][A-Za-z0-9_]+)|(\w+:\/\/\S+)",'', line_text)
    return line_text_without_username

def remove_stopwords(line_text):
    line_text_without_stopwords = ' '.join([word for word in line_text.split()
                                           if word not in (stopwords.words('english'))])
    return line_text_without_stopwords

def translate_abbreviations(line_text):
    text_list = line_text.split()
    with open("slang_abbreviations.txt", 'r') as my_csv_file:
        data_from_file = csv.reader(my_csv_file, delimiter="=")
        for index, _str in enumerate(text_list):
            _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
            for row in data_from_file:
                if _str.upper() == row[0]:
                    text_list[index] = row[1]
    final_string = ' '.join(text_list)
    return final_string

def remove_hashtags(line_text):
    line_text_without_hashtags = line_text.replace('#','')
    return line_text_without_hashtags

def lemmatizing(line_text):
    final_string = " ".join([word.lemma_ for word in en_core(line_text)])
    return final_string

def clean_line(line_text):
    text_zero = remove_numbers(line_text)
    text_one = remove_username(text_zero)
    text_two = remove_url(text_one)
    text_three = remove_punctuation(text_two)
    text_four = remove_hashtags(text_three)
    text_five = remove_stopwords(text_four)
    text_six = translate_abbreviations(text_five)
    text_final = lemmatizing(text_six)
    return text_final.lower()



Cleaning the text and saving cleaned version to new column 'cleaned_content'

In [ ]:
df_multiple = pd.read_csv("tweet_emotions.csv")
df_binary = pd.read_csv("tweet_emotions_binary.csv")
df_three_classes = pd.read_csv("tweet_emotions_three_classes.csv")

In [ ]:
df_multiple['cleaned_content'] = df_multiple.content.apply(clean_line)
df_binary['cleaned_content'] = df_binary.content.apply(clean_line)
df_three_classes['cleaned_content'] = df_three_classes.content.apply(clean_line)

Saving cleaned Dataframe to scv for future

In [ ]:
df_binary.to_csv('tweet_emotions_binary_cleaned.csv')
df_three_classes.to_csv('tweet_emotions_three_classes_cleaned.csv')
df_multiple.to_csv('tweet_emotions_cleaned.csv')

In [ ]:
df_binary

function which vectorize text and give the train and test data back

In [ ]:
def SpacyVec_data_from_df(df_input):
    df = df_input
    df['text_tokenized'] = df.clened_content.apply(lambda x: nlp_lg(x)) # tokenizing every text in the dataframe

    df.sentiment = pd.Categorical(df.sentiment) # changing type of a column to categorical
    df['Numeric_Labels'] = df.sentiment.cat.codes # saving category codes of column labels to new column

    #doing sentence embedding before splitting, because the order makes no difference
    df['text_vectorized'] = df['text_tokenized'].apply(lambda x: x.vector)

    # splitting data _________________COULD BE CHANGED____________________________________________________
    data_train, data_test, label_train, label_test = train_test_split(df.text_vectorized, df.Numeric_Labels,
                                                                 test_size=0.30,
                                                                 random_state=101, shuffle=True)
   # __________________________________________________________________________________________________________

    # appending all array rows into one and reshaping them from (1,300) shape to (300,1) shape
    data_list_train = [vec.reshape(1,-1) for vec in data_train]
    data_list_test = [vec.reshape(1,-1) for vec in data_test]

    # concentrating array to get them readable by sklearn models
    data_train_reshaped = np.concatenate(data_list_train)
    data_test_reshaped = np.concatenate(data_list_test)

    return data_train_reshaped, data_test_reshaped, label_train, label_test





In [ ]:
def TfIdf_data_from_df(df_input):
    df = df_input
    df.sentiment = pd.Categorical(df.sentiment) # changing type of a column to categorical
    df['Numeric_Labels'] = df.sentiment.cat.codes # saving category codes of column labels to new column

    # splitting data _________________COULD BE CHANGED____________________________________________________
    data_train, data_test, label_train, label_test = train_test_split(df.cleaned_content, df.Numeric_Labels, test_size=0.30, random_state=101, shuffle=True)
   # __________________________________________________________________________________________________________

   # vectorizing test and train data with Tfidf
    polarity_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
    polarity_bow_matrix_train = polarity_tfidf_vectorizer.fit_transform(data_train)
    polarity_bow_matrix_test = polarity_tfidf_vectorizer.transform(data_test)


    return polarity_bow_matrix_train, polarity_bow_matrix_test, label_train, label_test


In [ ]:
df_binary = pd.read_csv('tweet_emotions_binary_cleaned.csv')
data_train, data_test, label_train, label_test = TfIdf_data_from_df(df_binary)

In [ ]:
forest_classifier = RandomForestClassifier()

In [ ]:
forest_classifier = RandomForestClassifier()
forest_classifier.fit(data_train, label_train)
test_forest= forest_classifier.predict(data_test)

#evaluationg model and reporting accuracy and multi-class confusion matrix
print('\nEvaluate MLPClassifier:')
print('\nAccuracy: %.3f' % metrics.accuracy_score(label_test, test_forest))
print('\nConfusion Matrix: ')
multilabel_confusion_matrix(label_test, test_forest)

In [ ]:
def train_models(df, vectorizer, text="text", labels="emotion"):
    classifiers = [svm.LinearSVC(), RandomForestClassifier()] #DecisionTreeClassifier()]# # MLPClassifier()]

    #look into learning curves (do we have enough samples?) / validation curves (is the model under/overfitting?)

    X = vect.transform(df[text])
    y = df[labels]

    print("Amount of features: ", len(vect.get_feature_names_out()))
    print("Features: ", vect.get_feature_names_out())

    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    plot_learning_curve(classifiers, X, y, ylim=(0.5, 1.01), cv=cv, n_jobs=4)

    results = calc_metrics(classifiers, X, y)

    df_results = pd.DataFrame(results)
    return df_results

    plt.show(

In [ ]:
def train_models(df, vectorizer, text="text", labels="emotion"):
    classifiers = [svm.LinearSVC(), RandomForestClassifier()] #DecisionTreeClassifier()]# # MLPClassifier()]

    #look into learning curves (do we have enough samples?) / validation curves (is the model under/overfitting?)

    X = vect.transform(df[text])
    y = df[labels]

    print("Amount of features: ", len(vect.get_feature_names_out()))
    print("Features: ", vect.get_feature_names_out())

    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    plot_learning_curve(classifiers, X, y, ylim=(0.5, 1.01), cv=cv, n_jobs=4)

    results = calc_metrics(classifiers, X, y)

    df_results = pd.DataFrame(results)
    return df_results

    plt.show(